In [ ]:
# # 파일을 원하는 만큼 분할(쪼개기)
# f = open("/content/WAI-para_VB-1.csv", 'r')
# lines = f.readlines()
# f.close()
 
# cnt = 1
# fileName_Num = 1
 
 
# for line in lines:
#     fileName = "WAI-para_VB" + '_' +str(fileName_Num) + ".csv"
 
#     fw = open(fileName, "a")
#     fw.write(line)
#     fw.close()
 
#     # 목록을 5개씩 분할함
#     if cnt == 10:
#         fileName_Num = fileName_Num + 1
#         cnt = 0
    
#     cnt = cnt + 1
 

In [ ]:
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git
!pip install git+https://github.com/ssut/py-hanspell.git
!pip install konlpy
!pip install hangul_utils
!pip install git+https://github.com/boostcampaitech3/final-project-level3-nlp-03/eng_to_kor.git

In [ ]:
import requests
import json
import csv
import urllib.request
import pandas as pd

from tqdm import tqdm
from bs4 import BeautifulSoup
from hanspell import spell_checker
from konlpy.tag import Okt
from hangul_utils import join_jamos, split_syllables

# from eng_to_kor import eng_to_kor_func

In [ ]:
# def lang_check(word):
#     lang_client_id = "wrlba8fyci"
#     lang_client_secret = "2xAa0k7jK87UoaVgNLfgB0DaXuebyrCbxRzAst1Y"
#     # id, password 대신 파파고 언어감지 api를 발급받으면 제공되는 client id와 secret key를 대신 입력해주세요.
#     # https://developers.naver.com/docs/papago/papago-detectlangs-overview.md 에서 자세한 내용을 확인할 수 있습니다.

#     data = f"query={word}"
#     url = "https://openapi.naver.com/v1/papago/detectLangs"
#     request = urllib.request.Request(url)
#     request.add_header("X-NCP-APIGW-API-KEY-ID", lang_client_id)
#     request.add_header("X-NCP-APIGW-API-KEY", lang_client_secret)
#     response = urllib.request.urlopen(request, data=data.encode("utf-8"))
#     rescode = response.getcode()
#     if rescode == 200:
#         response_body = response.read()
#         source = json.loads(response_body.decode("utf-8"))["langCode"]
#         return source
#     else:
#         print("Error Code:" + rescode)


In [ ]:
# def sentence_to_word(word):
#     word = word.replace(".", "").replace("It's ", "")
#     word = word.replace("be ", "").replace("I ", "").replace("get ", "")
#     return word

In [ ]:

# def translate(word):
#     papago_translate_client_id = "wrlba8fyci"
#     papago_translate_client_secret = "2xAa0k7jK87UoaVgNLfgB0DaXuebyrCbxRzAst1Y"
#     # id, password 대신 파파고 NMT api를 발급받으면 제공되는 client id와 secret key를 대신 입력해주세요.
#     # https://developers.naver.com/docs/papago/papago-nmt-overview.md 에서 자세한 내용을 확인할 수 있습니다.

#     source = lang_check(word)
#     if source == "en":
#         target = "ko"
#     else:
#         source = "ko"
#         target = "en"
#     url = f"https://naveropenapi.apigw.ntruss.com/nmt/v1/translation"
#     data = f"source={source}&target={target}&text={word}"

#     request = urllib.request.Request(url)
#     request.add_header("X-NCP-APIGW-API-KEY-ID", papago_translate_client_id)
#     request.add_header("X-NCP-APIGW-API-KEY", papago_translate_client_secret)
#     response = urllib.request.urlopen(request, data=data.encode("utf-8"))
#     rescode = response.getcode()
#     if rescode == 200:
#         response_body = response.read()
#         result = json.loads(response_body.decode("utf-8"))["message"]["result"]
#         translated_word = result["translatedText"]
#         # if source == "en":
#         #     translated_word = sentence_to_word(translated_word)
#         return translated_word
#     else:
#         print("Error Code:" + rescode)

#     translated_word.to_csv(f"./translated_{csv_type}.csv")

In [ ]:
def find_antonyms(word):
    url = f"https://www.thesaurus.com/browse/{word}"

    response = requests.get(url)
    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, "html.parser")
        n_of_word = 2
        words = []
        for i in range(1, n_of_word + 1):
            word = soup.select(
                f"#antonyms > div.css-ixatld.e15rdun50 > ul > li:nth-of-type({i}) > a"    #nth 뒷부분 바꿔줬음
            )
            if str(word) == "[]":
                pass
            else:
                word = str(word).split("<!-- -->")
                word = word[0].split('">')[1]
            words.append(word)
        return words
    else:
        words = []
        return words
    words.to_csv(f"./antonyms_{csv_type}.csv")

In [ ]:
# No!!!!!이건 한국어 맞춤법 검사기였잖아!!!!!

# def spell_correct(word):
#     okt = Okt()
#     text = word
#     if "우다" in word:
#         text = word[:-2] + "ㅂ다"
#         text = split_syllables(text)
#         text = join_jamos(text)
#     hanspell_result = spell_checker.check(text)
#     text = hanspell_result.checked
#     return text.replace(" ", "")

In [ ]:
def make_dataframe(csv_type, is_da):
    df_name = f"WAI-{csv_type}-origin"
    csv_name = f"WAI-para_{csv_type}.csv"

    word_df = pd.DataFrame(
        columns=("original", "antonyms")
    )
    word_list = pd.read_csv(csv_name)
    for word in tqdm(word_list[csv_type]):
        # word = spell_correct(word)
        origin = word
        if not is_da:
            origin = word[:-1]
        # translated_original = translate(origin)
        translated_antonyms = find_antonyms(word)
        antonyms = []

        for word in translated_antonyms:
            if word:
                antonyms.append(word)
                # antonyms.append(eng_to_kor_func(word))
            else:
                antonyms.append("")
        new_data = {
            "original": origin,
            "antonyms": antonyms,
        }
        new_df = pd.DataFrame(new_data)
        word_df = pd.concat([word_df, new_df])
    word_df.to_csv(f"./{df_name}.csv")

In [ ]:
def main():
    #데이터셋이 동일한 폴더 안에 있어야 하는 코드로, 경로가 바뀐다면 조정이 필요합니다.
    origin = make_dataframe("VB", 1)

In [ ]:
if __name__ == "__main__":
    main()


In [ ]:
# # 실행한 파일 목록 제거

# import os


# for i in range(1,170):
#   file_path = f"/content/WAI-para_VB_{i}.csv"

#   if os.path.exists(file_path):
#     os.remove(file_path)